In [1]:
from datetime import date, datetime, timedelta
from timeit import default_timer as timer
from functools import reduce
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

import time
import pandas as pd
import numpy as np
import cx_Oracle
import os
import re

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)

<ipython-input-1-6215bfd8c668>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# 공통변수

TODAY = '20210409'
D_REG_KLID = '20210406'
STDAY = '20210310'
TODAY_DOT = '2021.04.09'

In [3]:
# DB connect

LOCATION = r"C:\instantclient_19_10"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
os.putenv('NLS_LANG', 'KOREAN_KOREA.KO16MSWIN949')

db_id = 'SIBUM'
# sibum 계정 패스워드 만료로 인해 변경(imsi00.! -> imsi0.!)
pw = 'imsi0.!'
host_name = '10.46.107.124:22751/SIBUMDB'
connection = cx_Oracle.connect(db_id, pw, host_name)

# 1. 전국수납_A07 (N01 ~ N10)

In [4]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_01_log.txt", "a")


print("\n-------------------------------------- BTS_01 프로그램 시작 --------------------------------------\n")

df1_timer_start = time.time() # 시작 타이머

df1_start = time.localtime() # 시작 시간

df1_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df1_start.tm_year, df1_start.tm_mon, df1_start.tm_mday, df1_start.tm_hour, df1_start.tm_min, df1_start.tm_sec)


# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT TPR_NO_ENC, PAY_DT AS 납부일자, SUM_RCP, D_REG_KLID, TAX_YY, TAX_MM, TAX_DIV, TAX_ITEM, C_SUNAP_IN  
FROM A07_SNP
WHERE
PAY_DT <= '{TODAY}'
AND D_REG_KLID = '{D_REG_KLID}'
"""

df1 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df1 = df1.loc[df1['D_REG_KLID']== D_REG_KLID]
df1 = df1.loc[df1['납부일자'] <= TODAY]

print("날짜계산", "---- 실행완료----")


today = datetime.strptime(TODAY, '%Y%m%d').date()

col_dict = dict()

for c in df1.columns:
    col_dict[c] = c.upper()
    
df1 = df1.rename(columns=col_dict)


def paid_before_due(row):
    due_as_mon = int(row['TAX_YY']) * 12 + int(row['TAX_MM']) + 1
    pay_str = row['납부일자']
    pay_as_mon = int(pay_str[0:4]) * 12 + int(pay_str[4:6])
    
    return pay_as_mon <= due_as_mon


df1['납기내_납부_여부'] = df1.progress_apply(paid_before_due, axis=1)

print("납기내_납부_여부 column 생성", "---- 실행완료----")

df1['납부일부터_경과일수'] = df1['납부일자'].progress_apply(lambda x: (today - datetime.strptime(x, '%Y%m%d').date()).days) 

df1 = df1.loc[df1['납부일부터_경과일수'] >= 0 ]

print("납부일부터_경과일수 column 생성", "---- 실행완료----")



# sub_지방세수납

df1_snp = df1

df1_snp_ybu = df1_snp.loc[df1_snp['납기내_납부_여부'] == True]

print("납기내_납부_여부 필터링", "---- 실행완료----")

df1_snp_ybu_1y = df1_snp_ybu.loc[df1_snp_ybu['납부일부터_경과일수'] <= 365]

df1_snp_ybu_3y = df1_snp_ybu.loc[df1_snp_ybu['납부일부터_경과일수'] <= 365*3]

print("납부일부터_경과일수 필터링", "---- 실행완료----")



# N01
#     N01_1년내 지방세 납기내 납부금액 : A07_SNP_AMT_DUE_1Y

print("N01을 만들기 위한 데이터 세트", "---- 실행완료----")

N01 = df1_snp_ybu_1y.groupby('TPR_NO_ENC')['SUM_RCP'].sum().reset_index()

N01 = N01.rename(columns = {'SUM_RCP': 'A07_SNP_AMT_DUE_1Y'})

print("N01생성", "---- 실행완료----")



# N02
#     N02_3년내 지방세 납기내 납부금액 : A07_SNP_AMT_DUE_3Y

print("N02을 만들기 위한 데이터 세트", "---- 실행완료----")

N02 = df1_snp_ybu_3y.groupby('TPR_NO_ENC')['SUM_RCP'].sum().reset_index()

N02 = N02.rename(columns = {'SUM_RCP': 'A07_SNP_AMT_DUE_3Y'})

print("N02생성", "---- 실행완료----")



# N03, N05
#     N03_1년내_정기분_납기내_납부금액 : A07_SNP_AMT_DUE_FIX_1Y
#     N05_1년내_신고분_납기내_납부금액 : A07_SNP_AMT_DUE_NOTI_1Y

df1_snp_ybu_1y_N0305 = df1_snp_ybu_1y.pivot_table('SUM_RCP', 'TPR_NO_ENC', 'TAX_DIV', aggfunc='sum').reset_index()

if '1' not in df1_snp_ybu_1y_N0305.columns:
    df1_snp_ybu_1y_N0305['1'] = 0

if '3' not in df1_snp_ybu_1y_N0305.columns:
    df1_snp_ybu_1y_N0305['3'] = 0    

df1_snp_ybu_1y_N0305 = df1_snp_ybu_1y_N0305.rename(columns = 
                                                   {'1': 'A07_SNP_AMT_DUE_FIX_1Y',
                                                    '3': 'A07_SNP_AMT_DUE_NOTI_1Y'})

print("N03, N05을 만들기 위한 데이터 세트", "---- 실행완료----")

N03 = df1_snp_ybu_1y_N0305[['TPR_NO_ENC', 'A07_SNP_AMT_DUE_FIX_1Y']]

print("N03생성", "---- 실행완료----")

N05 = df1_snp_ybu_1y_N0305[['TPR_NO_ENC', 'A07_SNP_AMT_DUE_NOTI_1Y']]

print("N05생성", "---- 실행완료----")



# N04, N06
#     N04_3년내_정기분_납기내_납부금액 : A07_SNP_AMT_DUE_FIX_3Y
#     N06_3년내_신고분_납기내_납부금액 : A07_SNP_CNT_DUE_FIX_3Y

df1_snp_ybu_3y_N0406 = df1_snp_ybu_3y.loc[df1_snp_ybu_3y['TAX_DIV'] == '1']

df1_snp_ybu_3y_N04 = df1_snp_ybu_3y_N0406[['TPR_NO_ENC', 'SUM_RCP']]

df1_snp_ybu_3y_N04 = df1_snp_ybu_3y_N04.rename(columns = {'SUM_RCP': 'A07_SNP_AMT_DUE_FIX_3Y'})

df1_snp_ybu_3y_N06 = df1_snp_ybu_3y_N0406[['TPR_NO_ENC']]

df1_snp_ybu_3y_N06['A07_SNP_CNT_DUE_FIX_3Y'] = 1

print("N04, N06을 만들기 위한 데이터 세트", "---- 실행완료----")

N04 = df1_snp_ybu_3y_N04.groupby('TPR_NO_ENC').sum().reset_index()

N04 = N04.fillna(0)

print("N04생성", "---- 실행완료----")

N06 = df1_snp_ybu_3y_N06.groupby('TPR_NO_ENC').sum().reset_index()

N06 = N06.fillna(0)

print("N06생성", "---- 실행완료----")



# N07
#     N07_3년내주민세_개인균등정기_납기내납부여부 : A07_SNP_DUE_JUMIN_EQ_3Y

df1_snp_ybu_3y_N07 = df1_snp_ybu_3y.loc[df1_snp_ybu_3y['TAX_ITEM'] == '104001']

print("N07을 만들기 위한 데이터 세트", "---- 실행완료----")

N07 = df1_snp_ybu_3y_N07[['TPR_NO_ENC']]

N07 = N07.drop_duplicates()

N07['A07_SNP_DUE_JUMIN_EQ_3Y'] = 1

print("N07생성", "---- 실행완료----")



# N08, N09
#     N08_가상계좌사용여부 : A07_SNP_IN_ACCT_VR_1Y
#     N09_신용카드사용여부 : A07_SNP_IN_CRCD_1Y

df1_snp_N0809 = df1_snp


def is_crcd(x):
    if len(x) > 60:
        return 0
    
    for c in ['410', '420', '430', '510', '740', '741', '742', '743', '744', '783', '901']:
        if x.startswith(c):
            return 1
    return 0


df1_snp_N0809['가상계좌사용'] = df1_snp['C_SUNAP_IN'].progress_apply(lambda x: 1 if x.startswith('250') else 0)

df1_snp_N0809['신용카드사용'] = df1_snp['C_SUNAP_IN'].progress_apply(lambda x: is_crcd(x))

df1_snp_N0809 = df1_snp_N0809[['TPR_NO_ENC', '신용카드사용', '가상계좌사용']].groupby('TPR_NO_ENC').sum().reset_index()

df1_snp_N0809 ['A07_SNP_IN_ACCT_VR_1Y'] = df1_snp_N0809 ['가상계좌사용'].progress_apply(lambda x: 1 if x > 0 else 0)

df1_snp_N0809 ['A07_SNP_IN_CRCD_1Y'] = df1_snp_N0809 ['신용카드사용'].progress_apply(lambda x: 1 if x > 0 else 0)

print("N08, N09을 만들기 위한 데이터 세트", "---- 실행완료----")

N08 = df1_snp_N0809[['TPR_NO_ENC', 'A07_SNP_IN_ACCT_VR_1Y']]

print("N08생성", "---- 실행완료----")

N09 = df1_snp_N0809[['TPR_NO_ENC', 'A07_SNP_IN_CRCD_1Y']]

print("N09생성", "---- 실행완료----")



# N10
#     N10_3년내 최근 납부일자로부터 경과월수 : A07_SNP_PAY_DT_MAX_MM_3Y

df1_snp_N10 = df1_snp[['TPR_NO_ENC', '납부일부터_경과일수']]

df1_snp_N10 = df1_snp_N10.loc[df1_snp_N10['납부일부터_경과일수'] < 365 * 3]

df1_snp_N10 = df1_snp_N10.groupby('TPR_NO_ENC').min().reset_index()

df1_snp_N10['A07_SNP_PAY_DT_MAX_MM_3Y'] = df1_snp_N10['납부일부터_경과일수'].progress_apply(lambda x: int( x / 30))

print("N10을 만들기 위한 데이터 세트", "---- 실행완료----")

N10 = df1_snp_N10[['TPR_NO_ENC', 'A07_SNP_PAY_DT_MAX_MM_3Y']]

print("N10생성", "---- 실행완료----")



# 프로그램1 최종변수 N01~N10 산출완료

df1_list = [N01, N02, N03, N04, N05, N06, N07, N08, N09, N10]

df1_total = reduce(lambda left, right: pd.merge(left, right, on= 'TPR_NO_ENC', how='outer'), df1_list)

display(df1_total)

print("최종변수 N01~N10 산출", "---- BTS_01 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df1, df1_snp, df1_snp_ybu, df1_snp_ybu_1y, df1_snp_ybu_3y, df1_snp_ybu_1y_N0305, df1_snp_ybu_3y_N0406, df1_snp_ybu_3y_N04, df1_snp_ybu_3y_N06, df1_snp_ybu_3y_N07, df1_snp_N0809, df1_snp_N10, df1_list, N01, N02, N03, N04, N05, N06, N07, N08, N09, N10

print("삭제된 데이터 리스트 :", "df1, df1_snp, df1_snp_ybu, df1_snp_ybu_1y, df1_snp_ybu_3y, df1_snp_ybu_1y_N0305, df1_snp_ybu_3y_N0406, df1_snp_ybu_3y_N04, df1_snp_ybu_3y_N06, df1_snp_ybu_3y_N07, df1_snp_N0809, df1_snp_N10, df1_list, N01, N02, N03, N04, N05, N06, N07, N08, N09, N10")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df1_end = time.localtime() # 종료 시간

df1_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df1_end.tm_year, df1_end.tm_mon, df1_end.tm_mday, df1_end.tm_hour, df1_end.tm_min, df1_end.tm_sec)


df1_timer_end = time.time() - df1_timer_start # 종료 타이머

times = str(timedelta(seconds=df1_timer_end)).split(".")

times = times[0]


print("\nBTS_01 프로그램 실행날짜 :", df1_start)

print("BTS_01 프로그램 종료날짜 :", df1_end)

print("BTS_01 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_01 프로그램 종료 --------------------------------------\n")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,TPR_NO_ENC,A07_SNP_AMT_DUE_1Y,A07_SNP_AMT_DUE_3Y,A07_SNP_AMT_DUE_FIX_1Y,A07_SNP_AMT_DUE_FIX_3Y,A07_SNP_AMT_DUE_NOTI_1Y,A07_SNP_CNT_DUE_FIX_3Y,A07_SNP_DUE_JUMIN_EQ_3Y,A07_SNP_IN_ACCT_VR_1Y,A07_SNP_IN_CRCD_1Y,A07_SNP_PAY_DT_MAX_MM_3Y
0,0000001c5b2dd0f99b0a60b0061ebc456f4516f7d69225...,729400.0,9234990.0,603860.0,720960.0,125540.0,5.0,NaN,1,1,6.0
1,00000c85876485ef6118ca15bbe0b95371a21dc70734b5...,12500.0,25000.0,12500.0,25000.0,NaN,2.0,1.0,1,0,7.0
2,00003b19901ad27c602215c5603659f6f3b61bd88d6f91...,1580640.0,6341900.0,1580640.0,5077680.0,NaN,14.0,1.0,1,1,3.0
3,000053118a460a987b87d351d1ce22f91a43516efcac24...,473920.0,1318420.0,473920.0,1318420.0,NaN,8.0,1.0,1,1,6.0
4,00005e70f82df3a951e081d286fd7356dc2890db2ec470...,11000.0,26000.0,11000.0,11000.0,NaN,1.0,1.0,1,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
1021421,fffe412b75b495bece500722021c3d00c8497e32b7cb74...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
1021422,fffeadbd77aceffd024df6b1c6a363ef794ff687ed35af...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN
1021423,fffec6bca30d45145ddba37d950dcc7a59c063cce6a29d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,21.0
1021424,fffefc33d3ed9140e0803117bf92843638c31666818c5e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN


# 3. 압류전국_A09 (N14 ~ N17, N288 ~ N290)

In [5]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_03_log.txt", "a")

print("-------------------------------------- BTS_03 프로그램 시작 --------------------------------------")

df3_timer_start = time.time() # 시작 타이머

df3_start = time.localtime() # 시작 시간

df3_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df3_start.tm_year, df3_start.tm_mon, df3_start.tm_mday, df3_start.tm_hour, df3_start.tm_min, df3_start.tm_sec)

# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT TPR_NO_ENC, LCHG_DT AS 최종변경일, SEIZ_DT AS 압류일자, SEIZ_CAL_DT AS 압류해지일자, SIDO_COD, SGG_COD, SEIZ_GDS_COD AS 압류물건코드, SEIZ_NO AS 압류일련번호, SEIZ_STP_RDT, D_REG_KLID
FROM A09_SEIZ 
WHERE 
D_REG_KLID = '{D_REG_KLID}'
"""

df3 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df3 = df3.loc[df3['D_REG_KLID']== D_REG_KLID]
df3 = df3.loc[(df3['압류일자'] <= TODAY) | (df3['최종변경일'] <= TODAY)]

print("날짜계산", "---- 실행완료----")


today = datetime.strptime(TODAY, '%Y%m%d').date()

def to_keep(row):
    condition_a = None
    condition_b = None
    
    seiz_cal_dt_str = row['압류해지일자']

    if seiz_cal_dt_str == None:
        condition_a = True
    else:
        if seiz_cal_dt_str.endswith('00') and len(seiz_cal_dt_str) == 8:
            seiz_cal_dt_str = seiz_cal_dt_str[:6] + '01'
        seiz_cal_dt = datetime.strptime(seiz_cal_dt_str, '%Y%m%d').date()
        condition_a = seiz_cal_dt > today
    
    LCHG_DT_str = row['최종변경일']

    SEIZ_STP_RDT = row['SEIZ_STP_RDT']
    
    if LCHG_DT_str == None or SEIZ_STP_RDT == None:
        condition_b = True
    else:
        if LCHG_DT_str.endswith('00') and len(LCHG_DT_str) == 8:
            LCHG_DT_str = LCHG_DT_str[:6] + '01'
        if SEIZ_STP_RDT.endswith('00') and len(SEIZ_STP_RDT) == 8:
            SEIZ_STP_RDT = SEIZ_STP_RDT[:6] + '01'
        LCHG_DT = datetime.strptime(LCHG_DT_str, '%Y%m%d').date()
        SEIZ_STP_RDT = datetime.strptime(SEIZ_STP_RDT, '%Y%m%d').date()
        condition_b = LCHG_DT <= today or SEIZ_STP_RDT <= today
    
    return condition_a and condition_b

df3['to_keep'] = df3.progress_apply(to_keep, axis=1)
df3 = df3.loc[df3['to_keep']]

print("to_keep column 생성", "---- 실행완료----")


col_dict = dict()

for c in df3.columns:
    col_dict[c] = c.upper()
    
df3 = df3.rename(columns=col_dict)

df3['경과일'] = df3['최종변경일'].progress_apply(lambda x: int((today - datetime.strptime(x[0:8], '%Y%m%d').date()).days))
df3 = df3.loc[df3['경과일'] >= 0]

print("경과일 column 생성", "---- 실행완료----")

# sub_압류

df3_seiz = df3



# N15
#     N15_1개월내압류등록건수 : A09_SEIZ_CNT_1M

df3_seiz_N15 = df3_seiz.loc[df3_seiz['경과일'] < 31]

df3_seiz_N15['A09_SEIZ_CNT_1M'] = 1

df3_seiz_N15 = df3_seiz_N15[['TPR_NO_ENC', 'A09_SEIZ_CNT_1M']]

print("N15을 만들기 위한 데이터 세트", "---- 실행완료----")

N15 = df3_seiz_N15.groupby('TPR_NO_ENC').sum().reset_index()

print("N15생성", "---- 실행완료----")



# N16
#     N16_압류진행시군구건수 : A09_SEIZ_CNT_SGG

df3_seiz_N16 = df3_seiz

df3_seiz_N16['시군구코드'] = df3_seiz_N16['SIDO_COD'] + df3_seiz_N16['SGG_COD']

df3_seiz_N16 = df3_seiz_N16[['TPR_NO_ENC', '시군구코드']]

df3_seiz_N16 = df3_seiz_N16.drop_duplicates()

print("N16을 만들기 위한 데이터 세트", "---- 실행완료----")

N16 = df3_seiz_N16.groupby('TPR_NO_ENC').count().reset_index()

N16 = N16.rename(columns={
    '시군구코드': 'A09_SEIZ_CNT_SGG'
})

print("N16생성", "---- 실행완료----")



# N17
#     N17_압류진행물건유형_차량 : A09_SEIZ_GDS_CAR

df3_seiz_N17 = df3_seiz[['TPR_NO_ENC', '압류물건코드']]

df3_seiz_N17 = df3_seiz_N17.loc[df3_seiz_N17['압류물건코드']=='04']

df3_seiz_N17= df3_seiz_N17.drop_duplicates()

print("N17을 만들기 위한 데이터 세트", "---- 실행완료----")

N17 = df3_seiz_N17[['TPR_NO_ENC']]

N17['A09_SEIZ_GDS_CAR'] = 1

print("N17생성", "---- 실행완료----")



# N288~N290
#     N288(285)_3년내압류진행대상여부 : A09_SEIZ_BOOL_3Y
#     N289(286)_3년내압류진행시군구건수 : A09_SEIZ_CNT_SGG_3Y
#     N290(287)_3년내압류진행물건유형_차량 : A09_SEIZ_GDS_CAR_3Y

# N289(286)
df3_seiz_3y = df3_seiz.loc[df3_seiz['경과일'] < 365*3]

df3_seiz_3y_N289 = df3_seiz_3y

df3_seiz_3y_N289['시군구코드'] = df3_seiz_3y_N289['SIDO_COD'] + df3_seiz_3y_N289['SGG_COD']

df3_seiz_3y_N289 = df3_seiz_3y_N289[['TPR_NO_ENC', '시군구코드']]

df3_seiz_3y_N289 = df3_seiz_3y_N289.drop_duplicates()

print("N289을 만들기 위한 데이터 세트", "---- 실행완료----")

N289 = df3_seiz_3y_N289.groupby('TPR_NO_ENC').count().reset_index()

N289 = N289.rename(columns={
    '시군구코드': 'A09_SEIZ_CNT_SGG_3Y'
})

print("N289생성", "---- 실행완료----")

# N290(287)
df3_seiz_3y_N290 = df3_seiz_3y[['TPR_NO_ENC', '압류물건코드']]

df3_seiz_3y_N290 = df3_seiz_3y_N290.loc[df3_seiz_3y_N290['압류물건코드']=='04']

df3_seiz_3y_N290 = df3_seiz_3y_N290.drop_duplicates()

print("N290을 만들기 위한 데이터 세트", "---- 실행완료----")

N290 = df3_seiz_3y_N290[['TPR_NO_ENC']]

N290['A09_SEIZ_GDS_CAR_3Y'] = 1

print("N290생성", "---- 실행완료----")

# N288(285)
df3_seiz_3y_N288 = N289.merge(N290, on='TPR_NO_ENC', how='outer')

df3_seiz_3y_N288['A09_SEIZ_BOOL_3Y'] = 1

print("N288을 만들기 위한 데이터 세트", "---- 실행완료----")

N288 = df3_seiz_3y_N288[['TPR_NO_ENC', 'A09_SEIZ_BOOL_3Y']]

print("N289생성", "---- 실행완료----")



# 프로그램3 최종변수 N14~N17, N288~N290 산출완료

df3_list = [N15, N16, N17, N288, N289, N290]

df3_total = reduce(lambda left, right: pd.merge(left, right, on= 'TPR_NO_ENC', how='outer'), df3_list)

# N14
#     N14_압류진행대상여부 : A09_SEIZ_BOOL

print("N14을 만들기 위한 데이터 세트", "---- 실행완료----")

df3_total['A09_SEIZ_BOOL'] = 1

print("N14생성", "---- 실행완료----")

df3_total = df3_total[[
'TPR_NO_ENC', 'A09_SEIZ_BOOL', 'A09_SEIZ_CNT_1M', 'A09_SEIZ_CNT_SGG', 'A09_SEIZ_GDS_CAR',
'A09_SEIZ_BOOL_3Y', 
'A09_SEIZ_CNT_SGG_3Y', 
'A09_SEIZ_GDS_CAR_3Y',  
]]

display(df3_total)

print("최종변수 N14~N17, N288~N290 산출", "---- BTS_01 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df3, df3_seiz, df3_seiz_N15, df3_seiz_N16, df3_seiz_N17, df3_seiz_3y, df3_seiz_3y_N289, df3_seiz_3y_N290, df3_seiz_3y_N288, df3_list, N15, N16, N17, N288, N289, N290

print("삭제된 데이터 리스트 :", "df3, df3_seiz, df3_seiz_N15, df3_seiz_N16, df3_seiz_N17, df3_seiz_3y, df3_seiz_3y_N289, df3_seiz_3y_N290, df3_seiz_3y_N288, df3_list, N15, N16, N17, N288, N289, N290")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df3_end = time.localtime() # 종료 시간

df3_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df3_end.tm_year, df3_end.tm_mon, df3_end.tm_mday, df3_end.tm_hour, df3_end.tm_min, df3_end.tm_sec)


df3_timer_end = time.time() - df3_timer_start # 종료 타이머

times = str(timedelta(seconds=df3_timer_end)).split(".")

times = times[0]


print("\nBTS_03 프로그램 실행날짜 :", df3_start)

print("BTS_03 프로그램 종료날짜 :", df3_end)

print("BTS_03 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_03 프로그램 종료 --------------------------------------\n")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,TPR_NO_ENC,A09_SEIZ_BOOL,A09_SEIZ_CNT_1M,A09_SEIZ_CNT_SGG,A09_SEIZ_GDS_CAR,A09_SEIZ_BOOL_3Y,A09_SEIZ_CNT_SGG_3Y,A09_SEIZ_GDS_CAR_3Y
0,00009f3517848b8b77e39132f39edce2dae71cb147b900...,1,1.0,1,1.0,1.0,1.0,1.0
1,0001012405297d80313c4f391e39aa5522d9f68036400a...,1,1.0,1,NaN,1.0,1.0,NaN
2,000168eef1a93a5a5a268f989fd0feecbc4ece2e21d1c0...,1,3.0,1,NaN,1.0,1.0,NaN
3,0001d2d7780bb9347ba180decc00c893632a02f73d74d6...,1,2.0,1,1.0,1.0,1.0,1.0
4,0002c29e61487f712487458b18bc5032764d30257e4c59...,1,1.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
392576,fffe65445e338d1e6cb0bfa64d7cfb85cd564b9fa22629...,1,NaN,2,1.0,1.0,1.0,1.0
392577,fffea3faf093c135341abdd79741e85eda715d62124f29...,1,NaN,2,1.0,NaN,NaN,NaN
392578,fffeecd4531ab8e7a7ac23d0e09622464450c70abbe6f0...,1,NaN,1,1.0,1.0,1.0,1.0
392579,ffff95b5c703494efedf8516c5eda1310772a713dd84e0...,1,NaN,1,1.0,1.0,1.0,1.0


# 4. 전국부동산_A10 (N18 ~ N31)

In [6]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_04_log.txt", "a")


print("-------------------------------------- BTS_04 프로그램 시작 --------------------------------------")

df4_timer_start = time.time() # 시작 타이머

df4_start = time.localtime() # 시작 시간

df4_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df4_start.tm_year, df4_start.tm_mon, df4_start.tm_mday, df4_start.tm_hour, df4_start.tm_min, df4_start.tm_sec)



# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT TPR_NO_ENC, DATA_GU, SIDO_COD, LDONG_COD, RI, SPCL, BONBUN, BUBUN, DONG, HO, ACQ_DT, SIGA_AMT, D_REG_KLID
FROM A10_BLD_LAND
WHERE D_REG_KLID = '{D_REG_KLID}'
"""

df4 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df4 = df4.loc[df4['D_REG_KLID']== D_REG_KLID]

print("날짜계산", "---- 실행완료----")

df4 = df4.drop_duplicates()

col_dict = dict()

for c in df4.columns:
    col_dict[c] = c.upper()

df4 = df4.rename(columns=col_dict)

df4 = df4.rename(columns={
    'DATA_GU': '자료구분',
    'ACQ_DT': '취득일자'
})

df4['cal_취득일자'] = df4['취득일자'].fillna('01010101')


def fixer(x):
    hangul = re.compile('[가-힣]+').findall(x)
    if len(hangul) > 0:
        x = '01010101'
        
    x = x.replace('-', '')
    x = x.replace(' ', '')
    
    for i in range(len(x), 8):
        n = 0 if i % 2 == 0 else 1
        x = f'{x}{n}'
    
    try:
        a = datetime.strptime(x, '%Y%m%d').date()
        return x
    
    except ValueError:
        return '01010101'
    
    
df4['cal_취득일자'] = df4['cal_취득일자'].progress_apply(lambda x: fixer(x))   

print("cal_취득일자 column 생성", "---- 실행완료----")

today = datetime.strptime(TODAY, '%Y%m%d').date()

df4['기준일자_이전_취득_여부'] = df4['cal_취득일자'].progress_apply(lambda x: datetime.strptime(x, '%Y%m%d').date() <= today)

print("기준일자_이전_취득_여부 column 생성", "---- 실행완료----")

df4_land_chi = df4.loc[df4['기준일자_이전_취득_여부'] == True]

print("기준일자_이전_취득_여부 필터링", "---- 실행완료----")



# N18, N19, N23, N24, N25, N26, N29, N30
#     N18_최근항공기시가표준합계 : A10_AIRP_PRC_SUM
#     N19_최근건물시가표준액합계 : A10_BD_PRC_SUM
#     N23_최근시섵물시가표준합계 : A10_FCLT_PRC_SUM
#     N24_최근주택시가표준합계 : A10_HOUSE_PRC_SUM
#     N25_최근토지시가표준합계 : A10_LAND_PRC_SUM
#     N26_최근재산시가표준합계(건물토지등) : A10_PRC_SUM_TOTAL
#     N29_최근부동산시가표준합계 : A10_REAL_ESTATE_PRC
#     N30_최근선박시가표준합계 : A10_SHIP_PRC_SUM

df4_land_chi_prc = df4_land_chi.pivot_table('SIGA_AMT', 'TPR_NO_ENC', '자료구분', aggfunc='sum').reset_index()

for c in ['01', '02', '03', '04', '05', '06']:
    if c not in df4_land_chi_prc.columns:
        df4_land_chi_prc[c] = 0

df4_land_chi_prc = df4_land_chi_prc.fillna(0)

df4_land_chi_prc['07'] = df4_land_chi_prc['01'] + df4_land_chi_prc['04'] + df4_land_chi_prc['05']
df4_land_chi_prc['08'] = df4_land_chi_prc['01'] + df4_land_chi_prc['02'] + df4_land_chi_prc['03'] + df4_land_chi_prc['04'] + df4_land_chi_prc['05'] + df4_land_chi_prc['06']

print("N18을 만들기 위한 데이터 세트", "---- 실행완료----")

N18 = df4_land_chi_prc[['TPR_NO_ENC', '03']]
N18 = N18.rename(columns = {'03': 'A10_AIRP_PRC_SUM'})

print("N18생성", "---- 실행완료----")

print("N19을 만들기 위한 데이터 세트", "---- 실행완료----")

N19 = df4_land_chi_prc[['TPR_NO_ENC', '01']]
N19 = N19.rename(columns = {'01': 'A10_BD_PRC_SUM'})

print("N19생성", "---- 실행완료----")

print("N23을 만들기 위한 데이터 세트", "---- 실행완료----")

N23 = df4_land_chi_prc[['TPR_NO_ENC', '06']]
N23 = N23.rename(columns = {'06': 'A10_FCLT_PRC_SUM'})

print("N23생성", "---- 실행완료----")

print("N24을 만들기 위한 데이터 세트", "---- 실행완료----")

N24 = df4_land_chi_prc[['TPR_NO_ENC', '05']]
N24 = N24.rename(columns = {'05': 'A10_HOUSE_PRC_SUM'})

print("N24생성", "---- 실행완료----")

print("N25을 만들기 위한 데이터 세트", "---- 실행완료----")

N25 = df4_land_chi_prc[['TPR_NO_ENC', '04']]
N25 = N25.rename(columns = {'04': 'A10_LAND_PRC_SUM'})

print("N25생성", "---- 실행완료----")

print("N26을 만들기 위한 데이터 세트", "---- 실행완료----")

N26 = df4_land_chi_prc[['TPR_NO_ENC', '08']]
N26 = N26.rename(columns = {'08': 'A10_PRC_SUM_TOTAL'})

print("N26생성", "---- 실행완료----")

print("N29을 만들기 위한 데이터 세트", "---- 실행완료----")

N29 = df4_land_chi_prc[['TPR_NO_ENC', '07']]
N29 = N29.rename(columns = {'07': 'A10_REAL_ESTATE_PRC'})

print("N29생성", "---- 실행완료----")

print("N30을 만들기 위한 데이터 세트", "---- 실행완료----")

N30 = df4_land_chi_prc[['TPR_NO_ENC', '02']]
N30 = N30.rename(columns = {'02': 'A10_SHIP_PRC_SUM'})

print("N30생성", "---- 실행완료----")


# 부동산만 주소결합

df4_land_chi_estate = df4_land_chi.loc[ (df4_land_chi['자료구분'] == '01')|(df4_land_chi['자료구분'] == '04') | (df4_land_chi['자료구분'] == '05')]

print("재산 중 부동산 데이터만 필터링", "---- 실행완료----")

df4_land_chi_estate['cal_address'] = df4_land_chi_estate['SIDO_COD'] + df4_land_chi_estate['LDONG_COD'] + df4_land_chi_estate['RI'] + df4_land_chi_estate['SPCL'] + df4_land_chi_estate['BONBUN'] + df4_land_chi_estate['BUBUN'] + df4_land_chi_estate['DONG']

print("부동산 데이터에 주소 결합", "---- 실행완료----")

df4_land_chi_estate['수도권'] = df4_land_chi_estate['SIDO_COD'].progress_apply(lambda x: x in ['11', '28', '41'])

print("수도권 여부 필터링", "---- 실행완료----")



# N20, N21
#     N20_최근주택건물소유건수 : A10_BLD_CNT
#     N21_최근수도권지역주택건물소유건수 : A10_BLD_SUDO_CNT

df4_land_chi_estate_N2021 = df4_land_chi_estate.loc[ (df4_land_chi_estate['자료구분'] == '01') | (df4_land_chi_estate['자료구분'] == '05')]

df4_land_chi_estate_N20 = df4_land_chi_estate_N2021[['TPR_NO_ENC', 'cal_address', '수도권']]

df4_land_chi_estate_N20 = df4_land_chi_estate_N20.drop_duplicates()

df4_land_chi_estate_N21 = df4_land_chi_estate_N20.loc[df4_land_chi_estate_N20['수도권']]

N20 = df4_land_chi_estate_N20.groupby('TPR_NO_ENC').count().reset_index()

N20 = N20[['TPR_NO_ENC', '수도권']]

N20 = N20.rename(columns={'수도권': 'A10_BLD_CNT'})

N21 = df4_land_chi_estate_N21.groupby('TPR_NO_ENC').count().reset_index()

N21 = N21[['TPR_NO_ENC', '수도권']]

N21 = N21.rename(columns={'수도권': 'A10_BLD_SUDO_CNT'})

N2021 = N20.merge(N21, on='TPR_NO_ENC', how='outer')

N2021 = N2021.fillna(0)

print("N20을 만들기 위한 데이터 세트", "---- 실행완료----")

N20 = N2021[['TPR_NO_ENC', 'A10_BLD_CNT']]

print("N20생성", "---- 실행완료----")

print("N21을 만들기 위한 데이터 세트", "---- 실행완료----")

N21 = N2021[['TPR_NO_ENC', 'A10_BLD_SUDO_CNT']]

print("N21생성", "---- 실행완료----")



# N22
#     N22_3년내 부동산 취득 시가표준액 수도권 : A10_CHI_AMT_REAST_SUDO_3Y

today = datetime.strptime(TODAY, '%Y%m%d').date()

df4_land_chi_estate_N22 = df4_land_chi_estate.loc[df4_land_chi_estate['수도권']]

df4_land_chi_estate_N22 = df4_land_chi_estate_N22[['TPR_NO_ENC', 'cal_취득일자', 'SIGA_AMT']]

df4_land_chi_estate_N22['취득일부터_경과일수'] = df4_land_chi_estate_N22['cal_취득일자'].progress_apply(lambda x: (today - datetime.strptime(x, '%Y%m%d').date()).days)

df4_land_chi_estate_N22 = df4_land_chi_estate_N22.loc[df4_land_chi_estate_N22['취득일부터_경과일수'] <= 365*3]

df4_land_chi_estate_N22 = df4_land_chi_estate_N22[['TPR_NO_ENC', 'SIGA_AMT']]

df4_land_chi_estate_N22 = df4_land_chi_estate_N22.fillna(0)

print("N22을 만들기 위한 데이터 세트", "---- 실행완료----")

N22 = df4_land_chi_estate_N22.groupby('TPR_NO_ENC').sum().reset_index()

N22 = N22.rename(columns={
  'SIGA_AMT': 'A10_CHI_AMT_REAST_SUDO_3Y'
})

print("N22생성", "---- 실행완료----")



# N27, 28, 31
#     N27_최근부동산소유여부 : A10_REAL_ESTATE_BOOL
#     N28_최근부동산소유건수 : A10_REAL_ESTATE_CNT
#     N31_최근선박시가표준합계 : A10_LAND_CNT

df4_land_chi_estate_N2831 = df4_land_chi_estate[['TPR_NO_ENC', 'cal_address', '자료구분']]

df4_land_chi_estate_N2831 = df4_land_chi_estate_N2831.drop_duplicates()

df4_land_chi_estate_N2831['A10_REAL_ESTATE_CNT'] = 1

df4_land_chi_estate_N2831['A10_LAND_CNT'] = df4_land_chi_estate_N2831['자료구분'].progress_apply(lambda x: 1 if x == '04' else 0)

N2831 = df4_land_chi_estate_N2831.groupby('TPR_NO_ENC').sum().reset_index()

print("N28을 만들기 위한 데이터 세트", "---- 실행완료----")

N28 = N2831[['TPR_NO_ENC', 'A10_REAL_ESTATE_CNT']]

print("N28생성", "---- 실행완료----")

print("N31을 만들기 위한 데이터 세트", "---- 실행완료----")

N31 = N2831[['TPR_NO_ENC', 'A10_LAND_CNT']]

print("N31생성", "---- 실행완료----")

print("N27을 만들기 위한 데이터 세트", "---- 실행완료----")

N27 = N2831['TPR_NO_ENC'].reset_index()

N27 = N27.drop(['index'], axis=1)

N27['A10_REAL_ESTATE_BOOL'] = 1

print("N27생성", "---- 실행완료----")



# 프로그램4 최종변수 N18~N31 산출완료

df4_list = [N18, N19, N20, N21, N22, N23, N24, N25, N26, N27, N28, N29, N30, N31]

df4_total = reduce(lambda left, right: pd.merge(left, right, on= 'TPR_NO_ENC', how='outer'), df4_list)

display(df4_total)

print("최종변수 N18~N31 산출", "---- BTS_04 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df4, df4_land_chi, df4_land_chi_prc, df4_land_chi_estate, df4_land_chi_estate_N2021, df4_land_chi_estate_N20, df4_land_chi_estate_N21, df4_land_chi_estate_N22, df4_land_chi_estate_N2831, N18, N19, N23, N24, N25, N26, N29, N30, N20, N21, N2021, N22, N2831, N28, N31, N27, df4_list

print("삭제된 데이터 리스트 :", "df4, df4_land_chi, df4_land_chi_prc, df4_land_chi_estate, df4_land_chi_estate_N2021, df4_land_chi_estate_N20, df4_land_chi_estate_N21, df4_land_chi_estate_N22, df4_land_chi_estate_N2831, N18, N19, N23, N24, N25, N26, N29, N30, N20, N21, N2021, N22, N2831, N28, N31, N27, df4_list")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df4_end = time.localtime() # 종료 시간

df4_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df4_end.tm_year, df4_end.tm_mon, df4_end.tm_mday, df4_end.tm_hour, df4_end.tm_min, df4_end.tm_sec)


df4_timer_end = time.time() - df4_timer_start # 종료 타이머

times = str(timedelta(seconds=df4_timer_end)).split(".")

times = times[0]


print("\nBTS_04 프로그램 실행날짜 :", df4_start)

print("BTS_04 프로그램 종료날짜 :", df4_end)

print("BTS_04 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_04 프로그램 종료 --------------------------------------\n")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,TPR_NO_ENC,A10_AIRP_PRC_SUM,A10_BD_PRC_SUM,A10_BLD_CNT,A10_BLD_SUDO_CNT,A10_CHI_AMT_REAST_SUDO_3Y,A10_FCLT_PRC_SUM,A10_HOUSE_PRC_SUM,A10_LAND_PRC_SUM,A10_PRC_SUM_TOTAL,A10_REAL_ESTATE_BOOL,A10_REAL_ESTATE_CNT,A10_REAL_ESTATE_PRC,A10_SHIP_PRC_SUM,A10_LAND_CNT
0,0000001c5b2dd0f99b0a60b0061ebc456f4516f7d69225...,0.0,392364279.0,2.0,2.0,393436279.0,0.0,74400000.0,7624000.0,474388279.0,1.0,6.0,474388279.0,0.0,4.0
1,000017749767a86b8e7cc7336a4d889cad59832e390f33...,0.0,0.0,1.0,1.0,NaN,0.0,178000000.0,680500.0,178680500.0,1.0,2.0,178680500.0,0.0,1.0
2,00003b19901ad27c602215c5603659f6f3b61bd88d6f91...,0.0,0.0,1.0,1.0,NaN,0.0,655000000.0,4620000.0,659620000.0,1.0,2.0,659620000.0,0.0,1.0
3,000053118a460a987b87d351d1ce22f91a43516efcac24...,0.0,0.0,1.0,1.0,NaN,0.0,363000000.0,3486000.0,366486000.0,1.0,2.0,366486000.0,0.0,1.0
4,0000a221bfa4cf9a4d3f25af1a76dd77ca1f2a27182381...,0.0,5850000.0,1.0,1.0,NaN,0.0,0.0,250600.0,6100600.0,1.0,3.0,6100600.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379658,ffff995940723839ac5b0b0acb4c8d327ff03df25313dd...,0.0,0.0,1.0,1.0,57331100.0,0.0,55000000.0,2331100.0,57331100.0,1.0,3.0,57331100.0,0.0,2.0
379659,ffff9fbda578d47b3083c93dcc801273ae0e613b4745d4...,0.0,0.0,1.0,1.0,NaN,0.0,59000000.0,1239000.0,60239000.0,1.0,2.0,60239000.0,0.0,1.0
379660,ffffae1ff39ddbc71030e8a9667162f9c677921721b58b...,0.0,0.0,1.0,1.0,NaN,0.0,107000000.0,1067000.0,108067000.0,1.0,2.0,108067000.0,0.0,1.0
379661,ffffc042f1c3f3c2bd771e488072b789acbee5a8330a41...,0.0,0.0,1.0,1.0,NaN,0.0,32531400.0,3260000.0,35791400.0,1.0,2.0,35791400.0,0.0,1.0


# 9. 월보수액_A05 (N65, 276, 287)

In [7]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_09_log.txt", "a")


print("-------------------------------------- BTS_09 프로그램 시작 --------------------------------------")

df9_timer_start = time.time() # 시작 타이머

df9_start = time.localtime() # 시작 시간

df9_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df9_start.tm_year, df9_start.tm_mon, df9_start.tm_mday, df9_start.tm_hour, df9_start.tm_min, df9_start.tm_sec)



# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT *
FROM A05_INCOME
WHERE D_REG_KLID <= '{D_REG_KLID}'
"""

df9 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df9 = df9.loc[df9['D_REG_KLID'] <= D_REG_KLID]

print("날짜계산", "---- 실행완료----")


today = datetime.strptime(TODAY, '%Y%m%d').date()


def how_many_days_passed(x):
    received_date = datetime.strptime(x + '-01', '%Y-%m-%d').date()
    return (today - received_date).days


df9['기준일자부터_경과일수'] = df9['RCV_YM'].progress_apply(lambda x: how_many_days_passed(x))

df9 = df9.loc[df9['기준일자부터_경과일수'] >= 0]

print("기준일자부터_경과일수 column 생성", "---- 실행완료----")

df9['월보수액_사용_가능여부'] = df9['기준일자부터_경과일수'].progress_apply(lambda x: x < 365*2)

print("월보수액_사용_가능여부 column 생성", "---- 실행완료----")


df9 = df9.sort_values(by=['기준일자부터_경과일수'], ascending=True)

df9 = df9.drop_duplicates(['TPR_NO_ENC'], keep='first')


# 14. KCB_etc (KCB) DB 연결
# 코드 통합시 호출 쿼리 제거 필요

print("KCB 데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT *
FROM B01_CB_CORRECTED
WHERE STDAY = '{STDAY}'
AND D_REG_KLID = '{D_REG_KLID}'
"""

df_kcb = pd.read_sql(query, con=connection)

print("KCB 데이터로드 성공", "---- 실행완료----")

df_kcb = df_kcb.loc[(df_kcb['D_REG_KLID']== D_REG_KLID) & (df_kcb['STDAY']== STDAY)]

print("날짜계산", "---- 실행완료----")


today = datetime.strptime(TODAY, '%Y%m%d').date()

col_dict = dict()

for c in df_kcb.columns:
    col_dict[c] = c.upper()
    
df_kcb = df_kcb.rename(columns=col_dict)

df_kcb = df_kcb.sort_values(by=['D_REG_KLID', 'IE30DSNAM'], ascending=[False, True])

df_kcb = df_kcb.drop_duplicates('TPR_NO_ENC', keep='first')


def is_he_dead(x):
    x = str(x).strip()
    if len(x) < 5:
        return 0
    else:
        dead_day = datetime.strptime(x, '%Y%m%d').date()
        if dead_day <= today:
            return 1
    return 0


df_kcb['A00_DEAD_YN'] = df_kcb['R20000002'].progress_apply(lambda x: is_he_dead(x))

print("A00_DEAD_YN column 생성", "---- 실행완료----")

df_kcb['mon_sal_as_won'] = df_kcb['IE30DSNAM'] / 12 * 10000

print("mon_sal_as_won column 생성", "---- 실행완료----")



# N65, N276(273), N287(284)
#     N65_최근월소득_건보소득과CB사소득보완  : A05_INCOME_CB
#     N276(273)_건강보험월소득 : A05_MON_SAL
#     N287(284)_건강보험월소득_수신년월 : A05_MON_SAL_RCV_YM

df9_kcb_mon_sal = df_kcb[['TPR_NO_ENC', 'mon_sal_as_won']]

df9_kcb_mon_sal = df9_kcb_mon_sal.merge(df9, on='TPR_NO_ENC', how='outer')

df9_kcb_mon_sal['월보수액_사용_가능여부'] = df9_kcb_mon_sal['월보수액_사용_가능여부'].fillna(False)


def wolbosu(row):
    if row['월보수액_사용_가능여부'] == True:
        return row['MON_SAL']
    else:
        return row['mon_sal_as_won']
    
    
df9_kcb_mon_sal['A05_INCOME_CB'] = df9_kcb_mon_sal.apply(wolbosu, axis=1)

df9_kcb_mon_sal = df9_kcb_mon_sal.rename(columns={
    'MON_SAL': 'A05_MON_SAL',
    'RCV_YM': 'A05_MON_SAL_RCV_YM',
})

# mon_sal_as_won, 월보수액_사용_가능여부 컬럼은 추후에 사용하지 않으나 브라이틱스 아웃풋 결과와 동일하게 하기 위해 사용

N00 = df9_kcb_mon_sal[['TPR_NO_ENC', 'mon_sal_as_won', '월보수액_사용_가능여부']]

print("N65, N276, N287을 만들기 위한 데이터 세트", "---- 실행완료----")

N65 = df9_kcb_mon_sal[['TPR_NO_ENC', 'A05_INCOME_CB']]

print("N65생성", "---- 실행완료----")

N276 = df9_kcb_mon_sal[['TPR_NO_ENC', 'A05_MON_SAL']]

print("N276생성", "---- 실행완료----")

N287 = df9_kcb_mon_sal[['TPR_NO_ENC', 'A05_MON_SAL_RCV_YM']]

print("N287생성", "---- 실행완료----")



# 프로그램9 최종변수 N65, N276, N287 산출완료

df9_list = [N287, N276, N00, N65,]

df9_total = reduce(lambda left, right: pd.merge(left, right, on= 'TPR_NO_ENC', how='outer'), df9_list)

display(df9_total)

print("최종변수 N65, N276, N287 산출", "---- BTS_09 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df9, df_kcb, df9_kcb_mon_sal, df9_list, N00, N65, N276, N287

print("삭제된 데이터 리스트 :", "df9, df_kcb, df9_kcb_mon_sal, df9_list, N00, N65, N276, N287")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df9_end = time.localtime() # 종료 시간

df9_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df9_end.tm_year, df9_end.tm_mon, df9_end.tm_mday, df9_end.tm_hour, df9_end.tm_min, df9_end.tm_sec)


df9_timer_end = time.time() - df9_timer_start # 종료 타이머

times = str(timedelta(seconds=df9_timer_end)).split(".")

times = times[0]


print("\nBTS_09 프로그램 실행날짜 :", df9_start)

print("BTS_09 프로그램 종료날짜 :", df9_end)

print("BTS_09 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_09 프로그램 종료 --------------------------------------")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,TPR_NO_ENC,A05_MON_SAL_RCV_YM,A05_MON_SAL,mon_sal_as_won,월보수액_사용_가능여부,A05_INCOME_CB
0,cc21ebbece77820edcd32e20c1065d9ff9dc1d95e14294...,NaN,NaN,833333.333333,False,8.333333e+05
1,26f96e2774e0bf4564c54a2e1e8cdcaa0ff195b9ddf421...,NaN,NaN,833333.333333,False,8.333333e+05
2,1eabfee7a25e713dcfb63fee1e77b9efc927b2863123a6...,NaN,NaN,833333.333333,False,8.333333e+05
3,51730461f10d0a9f99806a338499db357936a245f5db5b...,2019-10,2500000.0,833333.333333,True,2.500000e+06
4,af14c6dc16cce462060c464dcf7c3e4c44d5efb4367186...,2021-02,2258333.0,833333.333333,True,2.258333e+06
...,...,...,...,...,...,...
1330047,08a2f63a6bb5b5ad0c341185ab9215d8d368cdb4b8b070...,2015-01,1800000.0,NaN,False,NaN
1330048,3cc43c04ca30ff572d334aa76d540737d5e56fff6ca738...,2015-01,6231033.0,NaN,False,NaN
1330049,aa996435593e62145b7ed4d850ac2592fd0aa0ab7a699e...,2015-01,5500000.0,NaN,False,NaN
1330050,378ff053fd9a50553325539c94816246c39f60ac842f56...,2015-01,12055795.0,NaN,False,NaN


# 10. 소송_A19 (N83)

In [8]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_10_log.txt", "a")


print("\n-------------------------------------- BTS_010 프로그램 시작 --------------------------------------\n")

df10_timer_start = time.time() # 시작 타이머

df10_start = time.localtime() # 시작 시간

df10_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df10_start.tm_year, df10_start.tm_mon, df10_start.tm_mday, df10_start.tm_hour, df10_start.tm_min, df10_start.tm_sec)


# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT *
FROM A19_SUIT
WHERE D_REG_KLID <= '{D_REG_KLID}' AND D_REG_KLID > '20200206'
"""

df10 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df10 = df10.loc[(df10['D_REG_KLID'] <= D_REG_KLID) & (df10['D_REG_KLID'] > '20200206')]

print("날짜계산", "---- 실행완료----")


# 체납자명단_A01_CNP_LIST DB 연결
# 코드 통합시 호출 쿼리 제거 필요

print("체납자명단 데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT *
FROM A00_CNP_LIST
WHERE D_REG_KLID = '{D_REG_KLID}'
"""

df_cnp = pd.read_sql(query, con=connection)
df_cnp = df_cnp.drop_duplicates()

print("체납자명단 데이터로드 성공", "---- 실행완료----")

df_cnp_new = df_cnp[['TPR_NO_ENC', 'CD_SIDO', 'IS_NEW']]

df10_eve_end = df10.merge(df_cnp_new, on=['TPR_NO_ENC', 'CD_SIDO'], how='inner')

print("체납자명단과 데이터결합 성공", "---- 실행완료----")


def which_step(row):
    sim = -1
    if row['EVE_NO_4']:
        sim = 4
    elif row['EVE_NO_3']:
        sim = 3
    elif row['EVE_NO_2']:
        sim = 2    
    elif row['EVE_NO_1']:
        sim = 1
    else:
        sim = 0
    
    if row['D_EVE_END']:
        return f'{sim}-{row["D_REG_KLID"]}-{row["X_EVE_PRGS"]}-{row["D_EVE_END"]}'
    else:
        return f'{sim}-{row["D_REG_KLID"]}-{row["X_EVE_PRGS"]}-1111-11-11'
    
df10_eve_end['n심'] = df10_eve_end.progress_apply(which_step, axis=1)

print("n심 column 생성", "---- 실행완료----")

df10_eve_end = df10_eve_end.sort_values(by='n심', ascending=False)

df10_eve_end = df10_eve_end.drop_duplicates(['EVE_NO_1', 'TPR_NO_ENC'], keep='first')

df10_eve_end['D_EVE_END'] = df10_eve_end['D_EVE_END'].fillna('9999-12-12')


def suit_yn(row):
    if row['X_EVE_PRGS'] == '계류' and (row['D_EVE_END'] <= TODAY or row['D_EVE_END'] == '9999-12-12'):
        return 1
    else:
        return 0
    

df10_eve_end_N83 = df10_eve_end    

df10_eve_end_N83['A19_SUIT_YN'] = df10_eve_end.progress_apply(suit_yn, axis=1)

print("A19_SUIT_YN column 생성", "---- 실행완료----")

df10_eve_end_N83 = df10_eve_end_N83.loc[df10_eve_end_N83['A19_SUIT_YN'] == 1].sort_values(by='A19_SUIT_YN', ascending=False)

print("N83을 만들기 위한 데이터 세트", "---- 실행완료----")

N83 = df10_eve_end_N83.drop_duplicates('TPR_NO_ENC', keep='first')

print("N83생성", "---- 실행완료----")



# 프로그램10 최종변수 N83 산출완료

df10_total = N83

display(df10_total)

print("최종변수 N83 산출", "---- BTS_01 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df10, df_cnp, df_cnp_new, df10_eve_end, df10_eve_end_N83, N83

print("삭제된 데이터 리스트 :", "df10, df_cnp, df_cnp_new, df10_eve_end, df10_eve_end_N83, N83")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df10_end = time.localtime() # 종료 시간

df10_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df10_end.tm_year, df10_end.tm_mon, df10_end.tm_mday, df10_end.tm_hour, df10_end.tm_min, df10_end.tm_sec)


df10_timer_end = time.time() - df10_timer_start # 종료 타이머

times = str(timedelta(seconds=df10_timer_end)).split(".")

times = times[0]


print("\nBTS_10 프로그램 실행날짜 :", df10_start)

print("BTS_10 프로그램 종료날짜 :", df10_end)

print("BTS_10 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_10 프로그램 종료 --------------------------------------\n")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,C_SLF_ORG,EVE_NO_1,EVE_NO_2,EVE_NO_3,EVE_NO_4,X_EVE_PRGS,X_VICTORY,D_EVE_END,CD_SIDO,CN_SIDO,D_REG_KLID,TPR_NO_VLD,TPR_NO_ENC,TPR_NO_BIR,TPR_NO_GEN,IS_NEW,n심,A19_SUIT_YN
1714,41273 경기도 안산시 단원구,2016구합1678,2018누32691,None,None,계류,None,9999-12-12,41270,안산시03,20210406,True,804f02183351c2fa0661c24b28ecceddee2d52dea51783...,68,2,기존,2-20210406-계류-1111-11-11,1
2478,41390 경기도 시흥시,2017구합68715,2018누50347,None,None,계류,None,9999-12-12,41390,시흥시03,20210406,True,75949564441b16f0b09a9e6b679017a9f4f040a1485461...,65,1,기존,2-20210406-계류-1111-11-11,1
2427,41281 경기도 고양시덕양구,2019구합13126,2020누51480,None,None,계류,None,9999-12-12,41280,고양시03,20210406,True,375a27186dc70711dc6d90816ac47962281dbb1b2f1ae2...,58,1,기존,2-20210406-계류-1111-11-11,1
2888,41410 경기도 군포시,2020구합68708,None,None,None,계류,None,9999-12-12,41410,군포시03,20210406,True,76bf5edf16ef52ca86a3a137bc26198b2298405ecf6963...,43,1,기존,1-20210406-계류-1111-11-11,1
54,41590 경기도 화성시,2020구합76006,None,None,None,계류,None,9999-12-12,41590,화성시03,20210406,False,325f8fb8debf206c3d06030f4e0a7b9c079af5a0578951...,70,1,기존,1-20210406-계류-1111-11-11,1
2893,41287 경기도 고양시일산서구,2020구합14620,None,None,None,계류,None,9999-12-12,41280,고양시03,20210406,True,a6aca15789c35d9a2bab7f039e187bdd7a62d0545c2c67...,64,1,기존,1-20210406-계류-1111-11-11,1
2469,41135 경기도 성남시분당구,2015구합2551,None,None,None,계류,None,9999-12-12,41130,성남시03,20210406,True,fe7037d0479168fc0f869e9dc0ade56090b28e91598547...,61,2,기존,1-20210406-계류-1111-11-11,1


# 11. 체납자명단_A00 (N66, 71, 87)

In [9]:
# 로그폴더 지정 후 메모장 생성, 프린트 기록 시작

import sys

log_dir = './logs'

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

generate_month = date.today().strftime("%m") # 파일 이름에 쓰일 해당 월    
    
stdoutOrigin = sys.stdout
sys.stdout = open(f"./logs/[{generate_month}월]BTS_11_log.txt", "a")


print("\n-------------------------------------- BTS_11 프로그램 시작 --------------------------------------\n")

df11_timer_start = time.time() # 시작 타이머

df11_start = time.localtime() # 시작 시간

df11_start = "%04d.%02d.%02d %02d:%02d:%02d"%(df11_start.tm_year, df11_start.tm_mon, df11_start.tm_mday, df11_start.tm_hour, df11_start.tm_min, df11_start.tm_sec)


# 구현을 위한 임시 쿼리문

print("데이터로드 시작")

connection = cx_Oracle.connect(db_id, pw, host_name)

query = f"""
SELECT DISTINCT D_REG_KLID, TPR_NO_VLD, TPR_NO_ENC, TPR_NO_BIR, TPR_NO_GEN, IS_NEW 
FROM A00_CNP_LIST 
WHERE D_REG_KLID = '{D_REG_KLID}'
"""

df11 = pd.read_sql(query, con=connection)

print("데이터로드 성공", "---- 실행완료----")

df11 = df11.loc[df11['D_REG_KLID'] == D_REG_KLID]
df11 = df11.drop_duplicates()

print("날짜계산", "---- 실행완료----")

col_dict = dict()

for c in df11.columns:
    col_dict[c] = c.upper()

df11 = df11.rename(columns=col_dict)

this_year = int(TODAY[0:4])

df11['TPR_NO_BIR'] = df11['TPR_NO_BIR'].fillna('NO_DATA')
df11['TPR_NO_GEN'] = df11['TPR_NO_GEN'].fillna('NO_DATA')


df11_err_yn = df11

def is_valid_no(row):
    return 0 if row['TPR_NO_BIR'].isnumeric() and row['TPR_NO_GEN'].isnumeric() else 1

df11_err_yn['A00_TPR_NO_ERR_YN'] = df11_err_yn.progress_apply(is_valid_no, axis=1)

print("A00_TPR_NO_ERR_YN column 생성", "---- 실행완료----")


error_year = -999999999

def birth_year(row):
    b_year = None
    if row['A00_TPR_NO_ERR_YN'] == 1:
        b_year = error_year
    elif row['TPR_NO_GEN'] == '1' or row['TPR_NO_GEN'] == '2':
        b_year = int('19' + row['TPR_NO_BIR'])
    elif row['TPR_NO_GEN'] == '3' or row['TPR_NO_GEN'] == '4':
        b_year = int('20' + row['TPR_NO_BIR'])
    elif row['TPR_NO_GEN'] == '5' or row['TPR_NO_GEN'] == '6':
        b_year = int('19' + row['TPR_NO_BIR'])
    elif row['TPR_NO_GEN'] == '7' or row['TPR_NO_GEN'] == '8':
        b_year = int('20' + row['TPR_NO_BIR'])
    elif row['TPR_NO_GEN'] == '9' or row['TPR_NO_GEN'] == '0':
        b_year = int('18' + row['TPR_NO_BIR'])
    
    if b_year > this_year:
        return error_year
    
    return b_year

df11_err_yn['birth_year'] = df11_err_yn.progress_apply(birth_year, axis=1)

print("birth_year column 생성", "---- 실행완료----")

df11_err_yn['A00_AGE'] = df11_err_yn['birth_year'].apply(lambda x: this_year - x if x <= this_year and x != error_year else None)

print("A00_AGE column 생성", "---- 실행완료----")


def get_gender(x):
    if x in ['1', '3', '5', '7', '9']:
        return '남성'
    elif x in ['2', '4', '6', '8', '0']:
        return '여성'
    return '오류'


df11_err_yn['A00_GENDER'] = df11_err_yn['TPR_NO_GEN'].progress_apply(lambda x: get_gender(x))

print("A00_GENDER column 생성", "---- 실행완료----")

df11_err_yn_pi = df11_err_yn[['TPR_NO_ENC', 'A00_AGE', 'A00_GENDER', 'A00_TPR_NO_ERR_YN']]

df11_err_yn_pi = df11_err_yn_pi.drop_duplicates()

print("N66, N77, N87을 만들기 위한 데이터 세트", "---- 실행완료----")


# N66, N71(70), N87(84)
#     N66_연령 : A00_AGE
#     N71(70)_성별 : A00_GENDER
#     N87(84)_식별키오류여부 : A00_TPR_NO_ERR_YN

N66 = df11_err_yn_pi[['TPR_NO_ENC', 'A00_AGE']]

print("N66생성", "---- 실행완료----")

N71 = df11_err_yn_pi[['TPR_NO_ENC', 'A00_GENDER']]

print("N77생성", "---- 실행완료----")

N87 = df11_err_yn_pi[['TPR_NO_ENC', 'A00_TPR_NO_ERR_YN']]

print("N87생성", "---- 실행완료----")




# 프로그램11 최종변수 N66, N71, N87 산출완료

df11_list = [N66, N71, N87]

df11_total = reduce(lambda left, right: pd.merge(left, right, on= 'TPR_NO_ENC', how='outer'), df11_list)

display(df11_total)

print("최종변수 N66, N71, N87 산출", "---- BTS_11 실행완료----")

print("가상메모리 삭제시작")

# 메모리 삭제

import gc

del df11, df11_err_yn, df11_err_yn_pi, df11_list, N66, N71, N87

print("삭제된 데이터 리스트 :", "df11, f11_err_yn, df11_err_yn_pi, df11_list, N66, N71, N87")

print("가상메모리 용량 최적화", "---- 실행완료 ----")


df11_end = time.localtime() # 종료 시간

df11_end = "%04d.%02d.%02d %02d:%02d:%02d"%(df11_end.tm_year, df11_end.tm_mon, df11_end.tm_mday, df11_end.tm_hour, df11_end.tm_min, df11_end.tm_sec)


df11_timer_end = time.time() - df11_timer_start # 종료 타이머

times = str(timedelta(seconds=df11_timer_end)).split(".")

times = times[0]


print("\nBTS_11 프로그램 실행날짜 :", df11_start)

print("BTS_11 프로그램 종료날짜 :", df11_end)

print("BTS_11 프로그램 소요시간 :", times)

print("\n-------------------------------------- BTS_11 프로그램 종료 --------------------------------------\n")


# 프린트 기록 닫음

sys.stdout.close()
sys.stdout=stdoutOrigin

,TPR_NO_ENC,A00_AGE,A00_GENDER,A00_TPR_NO_ERR_YN
0,29ee2f90632282f125ec44b178fd779fd46630eeb74af0...,55.0,남성,0
1,de644466382067c08d1ad751320b0f78f894bb6c5eb48e...,72.0,남성,0
2,eb71f41645689f34e4385eb87b4ec3b731cd187b61a86e...,72.0,여성,0
3,03e3ebd73e6d0d43c2b4380e9577b178dd717be8234bc3...,72.0,남성,0
4,d207d9b8bcdc469682690f32297d5bb9e20d4b4b8c3acf...,62.0,여성,0
...,...,...,...,...
1214965,3c703ea11e254dd0a32ba56a975ab23478c0d9efbb3993...,53.0,남성,0
1214966,ec88b87a801dde46d30ad45ccf105c437f7c3a18b63abb...,52.0,남성,0
1214967,f77beec6f70208144848a5c00413ccecbe15fc4454a7e9...,44.0,남성,0
1214968,afa567a0379487b9f79c49dde7fc373d9f65e9082ad8ae...,42.0,여성,0
